# Boost Python

## Pedro Valiente

### @pvalienteverde


# Hola Mundo

**Caso:** Funcion sencilla sin argumentos y ningun valor de retorno

**WrapperFunciones/src/hola_mundo/ejemplo1.cpp**
```c++
#include <iostream>
#include <string>
#include <boost/python.hpp>

void HolaMundo() {
	std::cout << "Hola Mundo" << std::endl;
}

BOOST_PYTHON_MODULE(ejemplo1)
{
	boost::python::def("hola_mundo", HolaMundo,
			"Funcion que muestra nuestro primer saludo");
}
```
Codigo para compilar:

```bash
g++ -I/usr/include/python3.4m -c -std=c++14 -fPIC  -o ejemplo1.o ejemplo1.cpp
g++ -shared -lboost_python3 -o "ejemplo1.so"  ./ejemplo1.o
```

# std::vector <-> numpy 


**Caso:** Tenemos una funcion que multiplica un factor a cada elemento del vector:



**WrapperFunciones/src/vector/multiplicacion.cpp**
```c++
#include <vector>
#include <algorithm>
#include "multiplicacion.h"

std::vector<double> Multiplicacion(const std::vector<double> &datos, double factor) {
	std::vector<double> datos_transformados(datos);
	std::transform(datos_transformados.begin(), datos_transformados.end(),
			datos_transformados.begin(),
			[&factor](auto a) {return (a*factor);});
	return datos_transformados;
}
```

### 1 Solucion: Aproximacion por medio de una funcion auxiliar que convierta los tipos
* La funcion auxiliar *wrapper_multiplicacion* envuelve la funcion que queremos exponer

**WrapperFunciones/src/vector/ejemplo7.cpp**
```c++
#include <iostream>
#include <boost/python.hpp>
#include "boost/python/stl_iterator.hpp"
#include "boost/python/numeric.hpp"
#include <boost/assign/std/vector.hpp>
#include "multiplicacion.h"

namespace py = boost::python;

py::numeric::array wrapper_multiplicacion(const py::numeric::array& datos, double factor = 1) {
	py::stl_input_iterator<double> begin(datos), end;
	std::vector<double> datos_transformar(begin, end);
    
	auto nuevos_datos = Multiplicacion(datos_transformar, factor);
    
	py::list resultado;
	for (auto const &i : nuevos_datos)
		resultado.append(i);

	return py::numeric::array(resultado);
}

BOOST_PYTHON_FUNCTION_OVERLOADS(args_por_defecto_multiplicacion, wrapper_multiplicacion, 1, 2);
BOOST_PYTHON_MODULE(ejemplo7)
{
	py::numeric::array::set_module_and_type("numpy", "ndarray");
	py::def("wrapper_multiplicacion", wrapper_multiplicacion,args_por_defecto_multiplicacion());
}
```

In [100]:
import numpy as np

factor_multiplicacion=1.5
numero_de_elementos=5
vector = np.arange(numero_de_elementos,dtype=np.float64)
vector

array([ 0.,  1.,  2.,  3.,  4.])

In [101]:
import ejemplo7
ejemplo7.wrapper_multiplicacion(vector,factor_multiplicacion)

array([ 0. ,  1.5,  3. ,  4.5,  6. ])

### 2 Solucion: Aproximacion por medio del registro de tipos de conversion
* Por medio de la creacion de conversores explicitos, tanto de C++ a Python como viceversa
* **Ventajas**: Una vez definido los conversores, la extension del codigo c++ a python es directo. En mi [repo](https://github.com/pvalienteverde/MeetUpPyMadrid_Boost_Python/tree/master/WrapperClases/convertidores "Libreria de convertidores") está definidos los siguientes conversores: 

| **C++** | **Python** | **Notas**         
| :---------: |:---------:
|std::vector< double >| numpy | *En realidad cualquier clase iterable que se pueda construir por medio de iteradores y que la clase que contenga, tenga definido el constructor copia*
|boost::posix_time::time_duration|datetime.timedelta||
|boost::gregorian::date|datetime.date||
|boost::posix_time::ptime|datetime||
|boost::local_time::local_date_time|datetime|*Por ahora, unicamente reconoce la zona horaria UTC*




**WrapperFunciones/src/vector/ejemplo8.cpp**

```c++
#include <iostream>
#include <boost/python.hpp>
#include "multiplicacion.h"
#include "../../../WrapperClases/convertidores/include/envoltorio_objetos_iterables.h"

namespace py = boost::python;

BOOST_PYTHON_FUNCTION_OVERLOADS(args_por_defecto_multiplicacion, multiplicacion, 1, 2);
BOOST_PYTHON_MODULE(ejemplo8){
	convertidores::RegistrarObjetosIterables<std::vector<double>>();
	py::def("wrapper_multiplicacion", Multiplicacion);
}
```

In [102]:
import ejemplo8
ejemplo8.wrapper_multiplicacion(vector,factor_multiplicacion)

array([ 0. ,  1.5,  3. ,  4.5,  6. ])

# Wrappers De Clases / Libreria Externa

In [103]:
a=10
a

10